In [1]:
import numpy as np
import pandas as pd

In [2]:
# declare a list tasks whose products you want to use as inputs
upstream = None
product = None
min_rows_categories = None # minimo de objetos que devem conter essas categorias para ser desconsiderado
df_input_path = '../../dados/X_trainToronto.csv'

In [ ]:
df_reg = pd.read_csv(df_input_path)
df_reg = df_reg[['business_id', 'categories']]
df_reg

## Transformar categories em dummies

In [33]:
df_reg.categories = df_reg.categories.fillna('')
df_reg.categories = df_reg.categories.astype(str)
df_reg.categories = df_reg.categories.str.split(',')
df_reg.categories = df_reg.categories.apply(lambda x: [s.strip().lower() for s in x])
df_reg.categories

In [34]:
from collections import defaultdict

all_categories = defaultdict(int)
for c_list in df_reg.categories:
    if type(c_list) is float:
        c_list = []
    for c in c_list:
        all_categories[c.strip().lower()] = all_categories[c.strip().lower()] + 1

best_categories = pd.Series(all_categories)
best_categories = best_categories[best_categories > min_rows_categories]
others_categories = set(all_categories.keys()).difference(set(best_categories.index))
best_categories

wine bars                 127
tapas/small plates         94
restaurants              7418
bars                     1443
nightlife                1616
                         ... 
tires                      52
chicken shop               65
dog walkers                51
banks & credit unions      52
music venues              146
Length: 203, dtype: int64

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df_cat_dummies = pd.DataFrame(mlb.fit_transform(df_reg.categories),columns=mlb.classes_, index=df_reg.index)

if min_rows_categories > 0:
    df_cat_dummies['category_others_categories'] = df_cat_dummies.apply(lambda x: x[list(others_categories)].sum(), axis=1)
    df_cat_dummies = df_cat_dummies.drop(list(others_categories), axis=1)


df_reg = pd.concat([df_reg, df_cat_dummies], axis=1)
df_reg = df_reg.rename(columns={cat:'category_' + cat for cat in best_categories.index})
df_reg

In [ ]:
category_columns = [x for x in df_reg.columns if x.startswith('category_')]
df_reg[category_columns] = df_reg[category_columns].apply(lambda x: x.astype(float))

In [ ]:
df_reg.to_parquet(product['data'])

## Teste manual

In [ ]:
from IPython.display import display
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df_reg.iloc[13]['categories'])
    display(df_reg.iloc[[13]][category_columns])
    display('----------------------------------')
    display(df_reg.iloc[542]['categories'])
    display(df_reg.iloc[[542]][category_columns])
    display('----------------------------------')
    display(df_reg.iloc[5389]['categories'])
    display(df_reg.iloc[[5389]][category_columns])